Исходные данные

In [1]:
word_length = 125

Установка и вычисление контрольной суммы

In [2]:

from math import ceil, log2
from random import randint

In [29]:
def checksum(text):
    a = 0xFFFF
    b = 0xA001
    for byte in text:
        a ^= ord(byte)
        for i in range(8):
            last = a % 2
            a >>= 1
            if last == 1:
                a ^= b
    crc16 = hex(a).upper()
    return crc16

Утилитарные функции

In [30]:
def text_to_bits(text, encoding='utf-8', errors="ignore"):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))
  

def bits_to_text(bits, encoding='utf-8', errors="ignore"):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'

In [31]:
def calcRedundantBits(m):
	for i in range(m):
		if(2**i >= m + i + 1):
			return i

def posRedundantBits(data, r):
	j = 0
	k = 1
	m = len(data)
	res = ''
	for i in range(1, m + r+1):
		if(i == 2**j):
			res = res + '0'
			j += 1
		else:
			res = res + data[-1 * k]
			k += 1
	return res


def calcParityBits(arr, r):
	n = len(arr)
	for i in range(r):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		arr = arr[:n-(2**i)] + str(val) + arr[n-(2**i)+1:]
	return arr


def detectError(arr, nr):
	n = len(arr)
	res = 0
	for i in range(nr):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
		res = res + val*(10**i)
	return int(str(res), 2)


def remove_redundant_bits(data):
  r_idx = 1
  i_to_remove = []

  while r_idx+1 < len(data):
    i_to_remove.append(r_idx-1)
    r_idx *= 2

  i_to_remove = i_to_remove[::-1]

  for i in i_to_remove:
    data = data[:i] + data[i+1:]

  return data

Функции работы со словом

In [32]:
def encode_word(data):
  length = len(data)
  red_count = calcRedundantBits(length)
  data_with_zeros = posRedundantBits(data[::-1], red_count)
  data_with_pars = calcParityBits(data_with_zeros[::-1], red_count)
  return data_with_pars[::-1]


def decode_word(word_data):
  return remove_redundant_bits(word_data)


def decode_word_error(word_data):
  decoded_word_data = remove_redundant_bits(word_data)
  r = calcRedundantBits(len(decoded_word_data))
  idx_error = detectError(word_data[::-1], r) - 1
  if idx_error == -1 or idx_error >= len(word_data):
    return decoded_word_data, 0
  word_data = word_data[:idx_error] + ("0" if word_data[idx_error] == "1" else "1") + word_data[idx_error+1:]
  return remove_redundant_bits(word_data), 1

Функции работы с текстом

In [33]:
def encode(text, word_length):
  data = text_to_bits(text)
  data_length = len(data)
  num_blocks = ceil(data_length / word_length)
  data_blocks = [
      encode_word(data[i*word_length:(i+1)*word_length])
      for i in range(num_blocks)
  ]

  return data_blocks, checksum(text)


def decode(words_data):
  decoded_data = "".join(
      decode_word(word_data)
      for word_data in words_data
  )
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, checksum(decoded_text)


def decode_error(words_data):
  decoded_data = ""
  errors = 0
  for word_data in words_data:
    decoded_word, error = decode_word_error(word_data)
    decoded_data += decoded_word
    errors += error
  decoded_text = bits_to_text(decoded_data)
  return decoded_text, errors, checksum(decoded_text)

Функции добавления ошибок

In [34]:
def place_error(words_data, word_num, bit_num):
  word_data = words_data[word_num]
  word_data = word_data[:bit_num] + ("0" if word_data[bit_num] == "1" else "1") + word_data[bit_num+1:]
  words_data[word_num] = word_data
  return words_data


def place_random_errors(words_data, max_errors_per_word):
  errors_info = []
  broken_words = 0
  for word_num in range(len(words_data)):
    amount_of_errors  = randint(0, max_errors_per_word)
    broken_words += 1 if amount_of_errors else 0
    for err in range(amount_of_errors):
      error_idx = randint(0, len(words_data[word_num])-1)
      words_data = place_error(words_data, word_num, error_idx)
      errors_info.append([word_num, error_idx])
  return words_data, broken_words, errors_info

Основная функция

In [35]:
def main(text, word_length, max_broken_bits_per_word):
  encoded_blocks, check_init = encode(text, word_length)

  encoded_blocks, broken_words, errors_info = place_random_errors(encoded_blocks, max_broken_bits_per_word)
  print(f"Amount of broken words: {broken_words}")
  print(f"Amount of broken bits: {len(errors_info)}")

  decoded_text, check_dec = decode(encoded_blocks)
  print(f"Decoded text: {decoded_text}")
  print(f"Is the same? {decoded_text == text}")

  decoded_text_err, found_error_words, check_err = decode_error(encoded_blocks)
  print(f"Decoded text (fixing errors): {decoded_text_err}")
  print(f"Is the same? {decoded_text_err == text}")

  print(f"Found {found_error_words} broken words of {len(encoded_blocks)} total.")
  print(f"Found all errors? {len(errors_info) == found_error_words}")
  print(f"Decoding checksum: {check_init == check_dec}\nDecoding with errors fixed checksum: {check_init == check_err}")

Тесты

In [39]:
text = """Зона-02, TSRDEC1190292-Lambda-02 (Код Зоны) — это зона, расположенная в , в этой зоне происходит действие игры SCP: Secret Laboratory. В настоящий момент Зона-02, кажется, сосредоточена на изготовлении экспериментальных форм технологий для улучшения безопасности Фонда. Эти экспериментальные технологии могут включать в себя мощное оружие для оперативного подавления SCP или систему безопасности с ИИ для помощи при чрезвычайных ситуациях.

Зона-02 использует формат даты — ММ/дд/гг для всех локальных документов, когда для международных они используют — дд/ММ/гг. Хоть и Зона-02 использует формат даты ММ/дд/гг, распространённый в Серверной и Южной Америке, до сих пор неизвестно истинное местонахождение Зоны-02, так как эта информация засекречена, в отличие от Зоны-11 и Зоны-88, которые точно находятся в Северной и Южной Америке.

Во время теста с участием SCP-330, было выяснено, что конфеты обладают аномальными эффектами, которые можно получить, съев их. Эффекты со временем ослабевают. Во время теста, проходившего на Хэллоуин, было выяснено, что эффекты стали намного сильнее. После съедания экземпляра Розовой конфеты, Субъект D-330-4 немедленно , убив Д-р Эндрюса и серьёзно повредив камеру SCP-330. После этого SCP-330 был перемещён в LCZ-TC-01. C.A.S.S.I.E. впервые начал работу в Марте 2008-го года. Изначально он работал как простая Система громкой связи, хотя у него была ограниченная автономия, например, тушение пожаров или обнаружение вспышки заболевания в пределах комплекса. Последующие 6 лет C.A.S.S.I.E. постоянно улучшался с новыми и улучшенными системами, расширяющие управление C.A.S.S.I.E. над комплексом.

Нынешняя Роль C.A.S.S.I.E. заключается в оповещении персонала Зоны-02 о всевозможных чрезвычайных ситуациях, например о Нарушении Условий Содержания, пожарах, атаках Повстанцев Хаоса и прочих биологических угрозах. При обнаружении угрозы, C.A.S.S.I.E. оповещает главный персонал Комплекса и ждёт дальнейших указаний. При отсутствии указаний, C.A.S.S.I.E. будет действовать согласно заложенным протоколам и имеющимся возможностям для устранения угроз.

Из-за значения C.A.S.S.I.E. в жизни Комплекса, было введено множество мер безопасности. Ядро C.A.S.S.I.E. серьёзно охраняется, для получения доступа необходим Уровень Доступа 5. Некоторые источники сообщают, что последний раз Ядро C.A.S.S.I.E. было открыто в Декабре 2014 для установки обновлений. Подобные меры означают, что C.A.S.S.I.E. нельзя управлять удалённо - для предотвращения удалённого взлома и перехвата контроля над Комплексом.
Больше информации по C.A.S.S.I.E. можно найти здесь."""
main(text, word_length, max_broken_bits_per_word=1)

Amount of broken words: 134
Amount of broken bits: 134
Decoded text: ЗPна-02, TSRDEG1190292-Lambla-02 (Код Зоны) — это Чона, расположенна в , в этй зоне происходит дейفтвие игры SCP: Secret Laboratory. В настоящиҹ момент Зона-02, ка6ется, сощредоточPна на изготовлении экспериментальных(форм техԽологий для улучшения безопасноти Фонма. ЭтШ экспериментвльные технологии могут включать в себя мощное оружие для оператигного подвления SCP йли систему безопасности с ИИ для поPощи при чрѵзвычайных ситуациях.

Зжна-02 использует формат даты — ММ/дд/гг дPя всех локذльных докментов, когда дػя международных ни используют — дд/ММ/гг. Хот и Зона-02 использует фоЀмат даты ММ/дд+гг, распространённый в Серверной и Южной Америке, Фо сих пор Խеизвестно истинное местонхождение Зоны-02< так как эца информаنия засекреченд, в отличие от Зоны-11 и Зоны-88, которые точно находяђся в Севзрной и Южной Анерике.

Во врем теста с участием SCP-330, было выяснено, что конфеты"обладают аномальными эффектами, которые 